In [12]:
!pip install tabulate


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [15]:
!pip install openpyxl


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [16]:
import pandas as pd
from tabulate import tabulate
import os
import io
import subprocess

In [17]:
from openpyxl.utils.exceptions import IllegalCharacterError
import re

def clean_illegal_chars(text):
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0B-\x0C\x0E-\x1F]", "", text)
    return text


In [18]:
lista_codigos_cargos = "CCD,CCS,CYC,CYT,EAD,ECS,IAC,OCL,OCS,ODM,TCI,/PR,+3E,CCC,PEE,PRA,PRT,YCS,/3D,/3E,CMN,EDP,MKS,TLC"
lista_codigos_cargos = lista_codigos_cargos.split(",")
lista_codigos_cargos


['CCD',
 'CCS',
 'CYC',
 'CYT',
 'EAD',
 'ECS',
 'IAC',
 'OCL',
 'OCS',
 'ODM',
 'TCI',
 '/PR',
 '+3E',
 'CCC',
 'PEE',
 'PRA',
 'PRT',
 'YCS',
 '/3D',
 '/3E',
 'CMN',
 'EDP',
 'MKS',
 'TLC']

In [59]:
# lista_codigos_cargos_formateados = "*%20*".join(lista_codigos_cargos)
# lista_codigos_cargos_formateados
# lista_nombres_cargos = ['precep']
# lista_nombres_cargos_formateados = "%20".join(lista_nombres_cargos)
# lista_nombres_cargos_formateados

In [19]:
lista_distritos = ['berazategui','quilmes','avellaneda']
lista_distritos_formateados = "%20".join(lista_distritos)
lista_distritos_formateados

'berazategui%20quilmes%20avellaneda'

In [5]:
result = subprocess.run(f'''curl 'https://servicios3.abc.gob.ar/valoracion.docente/api/apd.oferta.encabezado/select?q=*%3A*&rows=10000&facet=true&facet.field=descdistrito&facet.field=descnivelmodalidad&facet.field=cargo&facet.field=estado&facet.limit=20000&facet.mincount=1&json.nl=map&fq=descdistrito%3A{lista_distritos_formateados}&fq=estado:Publicada&wt=csv' -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' -H 'Accept-Language: es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3' --compressed -H 'Connection: keep-alive' -H 'Cookie: _ga=GA1.3.1448501204.1598045085; _gid=GA1.3.1833139648.1598875839; IPCZQX034ee7c6fd=0100c3000a1432ac8ea31b21c67f7acb521691f8; ZNPCQ003-33383400=a89f2a51; ZNPCQ003-32383300=4e5b6dbe' -H 'Upgrade-Insecure-Requests: 1' | iconv -f iso-8859-1 -t UTF-8//TRANSLIT''', shell=True, capture_output=True, text=True)


In [20]:
df_sin_filtrar = pd.read_csv(io.StringIO(result.stdout))

In [21]:
df_sin_filtrar["areaincumbencia"] = df_sin_filtrar["areaincumbencia"].str.replace(" ","") #Por las dudas
df_sin_filtrar["areaincumbencia"] = df_sin_filtrar["areaincumbencia"].str.upper() #Por las dudas

In [22]:
df = df_sin_filtrar[df_sin_filtrar["areaincumbencia"].isin(lista_codigos_cargos)]

In [23]:
df

,estado,tipooferta,jornada,ige,miercoles,martes,acargodireccion,cuilautor,supl_hasta,turno,...,reemp_motivo,descdistrito,jueves,nivelmodalidad,viernes,descripcionarea,vuelta,descripcioncargo,ult_movimiento,_version_
0,Publicada,DESIGNACIONES DOCENTES,JS,3340149,08:00 A 12:30,08:00 A 12:30,No,27239663155,2025-05-26T03:00:00Z,M,...,Licencia medica(ART114medica),ENSENADA,08:00 A 12:30,A,08:00 A 12:30,PRECEPTOR (/PR),NaN,PRECEPTOR (/PR),2025-05-21T14:33:50.007Z,1832779512759189505
11,Publicada,DESIGNACIONES DOCENTES,JS,3340840,NaN,NaN,No,27269509118,2025-12-31T03:00:00Z,T,...,Relevo por servicios provisorios orden tecnico...,BERAZATEGUI,NaN,E,17:10 A 18:10,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-05-21T16:42:50.453Z,1832779515471855617
12,Publicada,DESIGNACIONES DOCENTES,JS,3341666,NaN,9:40 A 10:40,No,20275425894,2025-06-20T00:00:00Z,M,...,Licencia medica(ART114medica),BERAZATEGUI,9:40 A 10:40,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-05-21T16:17:26.927Z,1832779515475001345
16,Publicada,DESIGNACIONES DOCENTES,JS,3340225,7:30 A 12:00,7:30 A 12:00,No,27269509118,2025-06-19T03:00:00Z,M,...,Licencia medica(ART114medica),BERAZATEGUI,7:30 A 12:00,A,7:30 A 12:00,PRECEPTOR (/PR),NaN,PRECEPTOR (/PR),2025-05-21T14:44:47.917Z,1832779515484438531
50,Publicada,DESIGNACIONES DOCENTES,JS,3339962,13:50 a 18:20,13:50 a 18:20,No,24187813839,9999-12-30T17:14:35.560Z,T,...,NaN,AVELLANEDA,13:50 a 18:20,A,13:50 a 18:20,PRECEPTOR (/PR),NaN,PRECEPTOR (/PR),2025-05-21T14:14:11.147Z,1832779518835687424
84,Publicada,DESIGNACIONES DOCENTES,JS,3340997,12:50 A 17:20 HS,12:50 A 17:20 HS,No,20238653097,2025-05-26T00:00:00Z,T,...,Licencia medica(ART114medica),AVELLANEDA,12:50 A 17:20 HS,E,12:50 A 17:20 HS,PRECEPTOR (/PR),NaN,PRECEPTOR (/PR),2025-05-21T14:04:53.610Z,1832779518939496453
122,Publicada,DESIGNACIONES DOCENTES,JS,3339338,12.30 a 16.30,12.30 a 16.30,No,20248712520,2025-06-13T00:00:00Z,T,...,Licencia no medica por ARTICULO 114(ART114NOme...,QUILMES,12.30 a 16.30,K,12.30 a 16.30,PRECEPTOR (/PR),NaN,PRECEPTOR (/PR),2025-05-21T09:54:16.420Z,1832779520579469312
125,Publicada,DESIGNACIONES DOCENTES,JS,3340171,13.35 a 18.05,12.45 a 17.15,No,20248712520,2025-06-18T00:00:00Z,M,...,Licencia medica(ART114medica),QUILMES,13.35 a 18.05,E,12.45 a 17.15,PRECEPTOR (/PR),NaN,PRECEPTOR (/PR),2025-05-21T11:36:20.057Z,1832779520589955076
140,Publicada,DESIGNACIONES DOCENTES,JE,3340606,8.00 a 16.00,8.00 a 16.00,No,20248712520,2025-05-23T03:00:00Z,MT,...,Licencia medica(ART114medica),QUILMES,8.00 a 16.00,E,8.00 a 16.00,PRECEPTOR (/PR),NaN,PRECEPTOR (/PR),2025-05-21T15:55:12.803Z,1832779520623509514
146,Publicada,DESIGNACIONES DOCENTES,JS,3341808,08:20- 12:50 HS,08:20- 12:50 HS,No,23219778694,2025-06-06T00:00:00Z,M,...,Licencia medica(ART114medica),QUILMES,008:20- 12:50 HS,E,08:20- 12:50 HS,PRECEPTOR (/PR),NaN,PRECEPTOR (/PR),2025-05-21T16:52:45.267Z,1832779520635043845


In [25]:
df.columns

Index(['estado', 'tipooferta', 'jornada', 'ige', 'miercoles', 'martes',
       'acargodireccion', 'cuilautor', 'supl_hasta', 'turno', 'idoferta',
       'domiciliodesempeno', 'cuilganador', 'sabado', 'listadoorigenganador',
       'id', 'iddetalle', 'cargo', 'tomaposesion', 'supl_revista',
       'postulacion_idganador', 'reemp_apeynom', 'numdistrito',
       'areaincumbencia', 'nombreganador', 'finoferta', 'observaciones',
       'cupof', 'tipooferta_id', 'supl_desde', 'reemp_cuil', 'escuela',
       'fechadesignacion', 'iniciooferta', 'puntajeganador', 'prioridad',
       'hsmodulos', 'cursodivision', 'idsuna', 'descnivelmodalidad', 'lunes',
       'timestamp', 'infectocontagiosa', 'reemp_motivo', 'descdistrito',
       'jueves', 'nivelmodalidad', 'viernes', 'descripcionarea', 'vuelta',
       'descripcioncargo', 'ult_movimiento', '_version_'],
      dtype='object')

In [29]:
df["domiciliodesempeno"] = df["domiciliodesempeno"].apply(clean_illegal_chars)

/tmp/ipykernel_6856/3741950650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["domiciliodesempeno"] = df["domiciliodesempeno"].apply(clean_illegal_chars)


In [30]:
df.to_excel("lista_ofertas.xlsx",index=False)


In [41]:
# files = os.listdir("./DATA/")
# files
# files = os.listdir("./DATA/")
# files.sort(reverse=True)
# file = files[0] #me quedo con el último
# csv_file = f'./DATA/{file}'  # Replace with the path to your CSV file
# df = pd.read_csv(csv_file)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 3 to 252
Data columns (total 53 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   estado                 27 non-null     object 
 1   tipooferta             27 non-null     object 
 2   jornada                27 non-null     object 
 3   ige                    27 non-null     int64  
 4   miercoles              10 non-null     object 
 5   martes                 9 non-null      object 
 6   acargodireccion        27 non-null     object 
 7   cuilautor              27 non-null     int64  
 8   supl_hasta             27 non-null     object 
 9   turno                  27 non-null     object 
 10  idoferta               27 non-null     int64  
 11  domiciliodesempeno     27 non-null     object 
 12  cuilganador            0 non-null      float64
 13  sabado                 0 non-null      float64
 14  listadoorigenganador   0 non-null      float64
 15  id          

In [83]:
orden_distritos = {'QUILMES':2,'AVELLANEDA':1,'BERAZATEGUI':0}
ofertas_por_distrito = df.groupby(['descdistrito','descripcioncargo'])['ige'].nunique().reset_index()
ofertas_por_distrito.columns = ['Distrito','Cargo','Cantidad']
ofertas_por_distrito["order"] = ofertas_por_distrito.Distrito.map(orden_distritos)
ofertas_por_distrito = ofertas_por_distrito.sort_values(by=['order','Cantidad'],ascending=False)
ofertas_por_distrito.drop(columns="order",inplace=True)
ofertas_por_distrito = tabulate(
    ofertas_por_distrito, 
    headers=ofertas_por_distrito.columns, 
    tablefmt="html",
    showindex=False,
)
ofertas_por_distrito


Distrito,Cargo,Cantidad
QUILMES,CONSTRUCCION DE CIUDADANIA (CCD),2
QUILMES,PRECEPTOR (/PR),2
BERAZATEGUI,CONSTRUCCION DE CIUDADANIA (CCD),21
BERAZATEGUI,"COMUNICACIÓN, CULTURA Y SOCIEDAD (CCS)",1
BERAZATEGUI,PRECEPTOR (/PR),1


In [84]:
ofertas_por_escuela = df.groupby(['descdistrito','escuela','domiciliodesempeno','descripcioncargo'])['ige'].nunique().reset_index()
ofertas_por_escuela.columns = ['Distrito','Escuela','Dirección','Cargo','Cantidad']
ofertas_por_escuela["order"] = ofertas_por_escuela.Distrito.map(orden_distritos)
ofertas_por_escuela = ofertas_por_escuela.sort_values(by=['order','Cantidad'],ascending=False)
ofertas_por_escuela.drop(columns="order",inplace=True)
ofertas_por_escuela = tabulate(
    ofertas_por_escuela, 
    headers=ofertas_por_escuela.columns, 
    tablefmt="html",
    showindex=False,
)
ofertas_por_escuela

Distrito,Escuela,Dirección,Cargo,Cantidad
QUILMES,0085JI0929,856 E/ 883 Y 884,PRECEPTOR (/PR),1
QUILMES,0085MS0003,CORRIENTES ESQ QUINTANA,CONSTRUCCION DE CIUDADANIA (CCD),1
QUILMES,0085MS0016,CALCHAQUI Y BOMBERO SANCHEZ,CONSTRUCCION DE CIUDADANIA (CCD),1
QUILMES,0085MS0024,880 esquina 833,PRECEPTOR (/PR),1
BERAZATEGUI,0119MS0023,163 E/ 23 Y 24,CONSTRUCCION DE CIUDADANIA (CCD),3
BERAZATEGUI,0119MS0005,148,CONSTRUCCION DE CIUDADANIA (CCD),2
BERAZATEGUI,0119MS0018,27 E/ 106 Y 107,CONSTRUCCION DE CIUDADANIA (CCD),2
BERAZATEGUI,0119MS0022,129,CONSTRUCCION DE CIUDADANIA (CCD),2
BERAZATEGUI,0119MS0002,304,CONSTRUCCION DE CIUDADANIA (CCD),1
BERAZATEGUI,0119MS0005,148,PRECEPTOR (/PR),1


In [85]:
cantidad_ofertas = df["ige"].nunique()
cantidad_ofertas

27